## Chains
链允许我们将多个组件组合在一起，以创建一个单一的、连贯的应用程序

In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
import httpx
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0.0,api_key=os.environ['OPENAI_API_KEY'],base_url="https://api-inference.modelscope.cn/v1/", http_client=httpx.Client(verify=False),model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B")

In [5]:
import pandas as pd
df = pd.read_csv('Data.csv',encoding='utf-8')

In [6]:
df

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...
5,L'Or Espresso Café \n,Je trouve le goût médiocre. La mousse ne tient...
6,Hervidor de Agua Eléctrico,"Está lu bonita calienta muy rápido, es muy fun..."


### 1. LLMChain
LLMChain是一个简单但非常强大的链，也是后面我们将要介绍的许多链的基础。

In [8]:
from langchain.prompts import ChatPromptTemplate   #导入聊天提示模板
from langchain.chains import LLMChain    #导入LLM链。
import warnings
warnings.filterwarnings('ignore')
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)
chain = LLMChain(llm=llm, prompt=prompt)
chain.run(product="Queen Size Sheet Set")

'**QueenCloud**\n\n**Rationale:**\n- **Clarity and Simplicity:** Clearly indicates the product size (Queen) and evokes imagery of comfort and luxury.\n- **Memorable Name:** The name "QueenCloud" is catchy and easy to remember, capturing the essence of sophistication and comfort.\n- **Aesthetic Appeal:** Combines "Queen" with "Cloud," suggesting softness and breathability, which are desirable qualities in bedding.\n- **Potential for Branding:** The name is versatile for branding, with room to incorporate design elements that reflect luxury or style.\n\nThis name effectively communicates the product\'s characteristics while appealing to a premium audience.'

### 2. SimpleSequentialChain
顺序链（Sequential Chains）是按预定义顺序执行其链接的链。具体来说，我们将使用简单顺序链（SimpleSequentialChain），这是顺序链的最简单类型，其中每个步骤都有一个输入/输出，一个步骤的输出是下一个步骤的输入

In [18]:
from langchain.chains import SimpleSequentialChain
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?,only output name"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt)
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{name}"
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)
simple_chain = SimpleSequentialChain(chains=[chain_one,chain_two],verbose=True)
product = "Queen Size Sheet Set"
simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Luxury Queen Sheets
Experience luxury with our elegant and stylish luxury sheets for your bedroom, crafted with premium materials for unparalleled comfort and sophistication.

> Finished chain.


'Experience luxury with our elegant and stylish luxury sheets for your bedroom, crafted with premium materials for unparalleled comfort and sophistication.'

### 3. SequentialChain
当只有一个输入和一个输出时，简单的顺序链可以顺利完成。但是当有多个输入或多个输出时该如何实现呢？
我们可以使用普通的顺序链来实现这一点

In [31]:
from langchain.chains import SequentialChain
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to chinese:"
    "\n\n{Review}"
)
#chain 1: 输入：Review 输出： 中文的 Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="Chinese_Review"
                    )
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{Chinese_Review}"
)
# chain 2: 输入：中文的Review   输出：summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review},you need only output the language"
)
# chain 3: 输入：Review  输出：language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="raw_language"
                      )
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {raw_language}"
)
# chain 4: 输入： summary, language    输出： 后续回复
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["Chinese_Review","raw_language", "summary","followup_message"],
    verbose=True
)
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'Chinese_Review': '我尝过同款，感觉味道更好呢。  \n老货还是假货？',
 'raw_language': 'French',
 'summary': '用户试用后表示味道更佳，但对产品真伪存疑。',
 'followup_message': "Merci beaucoup pour votre retour positif ! Nous sommes ravis de constater que le parfum semble meilleur pour vous. Cependant, pour répondre à votre question concernant la genuineité du produit, nous devrions demander plus de détails. La qualité et la réalitée du parfum sont importants pour nous, et nous aimons garantir que chaque produit que nous offrons est de qualité. Si vous avez des questions spécifiques ou si vous avez besoin d'aide pour confirmer la réalitée du produit, n'hésitez pas à nous le dire !"}

### 4. LLMRouterChain（此链使用 LLM 来确定如何路由事物）
我们需要一个多提示链。这是一种特定类型的链，用于在多个不同的提示模板之间进行路由。 但是，这只是你可以路由的一种类型。你也可以在任何类型的链之间进行路由

In [32]:
#第一个提示适合回答物理问题
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


#第二个提示适合回答数学问题
math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""


#第三个适合回答历史问题
history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


#第四个适合回答计算机问题
computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [33]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [34]:
from langchain.chains.router import MultiPromptChain  #导入多提示链
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [35]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [36]:
destinations_str

'physics: Good for answering questions about physics\nmath: Good for answering math questions\nHistory: Good for answering history questions\ncomputer science: Good for answering computer science questions'

In [37]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [38]:
MULTI_PROMPT_ROUTER_TEMPLATE = """
Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [41]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [42]:
router_template

'\nGiven a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revisingit will ultimately lead to a better response from the language model.\n\n<< FORMATTING >>\nReturn a markdown code snippet with a JSON object formatted to look like:\n```json\n{{\n    "destination": string \\ name of the prompt to use or "DEFAULT"\n    "next_inputs": string \\ a potentially modified version of the original input\n}}\n```\n\nREMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is notwell suited for any of the candidate prompts.\nREMEMBER: "next_inputs" can just be the original input if you don\'t think any modifications are needed.\n\n<< CANDIDATE PROMPTS >>\nphysics: Good for answering questions about physics\nmath: Good for answering math

In [43]:
#多提示链
chain = MultiPromptChain(router_chain=router_chain,    #l路由链路
                         destination_chains=destination_chains,   #目标链路
                         default_chain=default_chain,      #默认链路
                         verbose=True   
                        )

In [44]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation is the theoretical radiation emitted by an idealized object, known as a black body, which absorbs all incident electromagnetic radiation. This perfect absorber emits radiation across all wavelengths, with the intensity depending on its temperature. Key aspects of black body radiation include:\n\n1. **Definition**: A black body absorbs all incident light and emits radiation based on its temperature. It is a conceptual model, as real objects may reflect some light.\n\n2. **Spectrum**: The emitted radiation's spectrum follows the Planck distribution, which describes energy distribution across wavelengths. The peak wavelength is inversely proportional to temperature (Wien's displacement law), and the total power radiated is proportional to the fourth power of temperature (Stefan-Boltzmann law).\n\n3. **Applications**: Used in thermal design, astrophysics (e.g., star emission), and communication systems. It helps explain phenomena like the color of stars and the therma

In [45]:
chain.run("黑洞是种什么物质?")



> Entering new MultiPromptChain chain...
physics: {'input': '黑洞是种什么物质?'}
> Finished chain.


'黑洞并不是一种传统意义上的物质，而是一种极端的天体现象，其本质是时空的极度弯曲所导致的极端引力场。以下是关于黑洞的更准确解释：\n\n1. **定义与预测**：\n   - 黑洞是广义相对论中预测的极端引力体，由大质量恒星在生命末期坍缩形成。\n   - 它是一个引力极强的区域，使得光和 matter 无法逃脱其引力束缚，形成所谓的“事件视界”。\n\n2. **内部结构**：\n   - 黑洞的核心是一个奇点，理论认为这里时空曲率变得无限大，物质和能量被无限压缩。\n   - 奇点是一个数学上的概念，实际是否存在尚有争议，物理学家仍在探索其内部结构。\n\n3. **组成与观测**：\n   - 黑洞本身并不含有传统意义上的物质，而是由极强的引力场主导的空间和时间结构。\n   - 黑洞的质量主要来自于其父恒星的质量，而非内部物质的组成。\n   - 黑洞通过吞噬其他物质和光来增重，但其自身的“物质”含量尚不清楚，因为未直接观测到其内部。\n\n4. **暗物质与黑洞的关系**：\n   - 黑洞与暗物质的关系尚未明确，暗物质是解释宇宙结构的动力学的重要理论概念，而黑洞是引力极端的表现。\n   - 黑洞可能与暗物质相互作用，但目前没有直接证据证明它们直接相关。\n\n5. **研究与探索**：\n   - 科学家通过观测黑洞周围的影响，如引力扭曲、X射线喷流等，来研究其性质。\n   - 黑洞的存在 challenge 了我们的理解，促使我们探索更多关于时空和引力的新理论。\n\n总结来说，黑洞是一种极端引力体，由极强的引力场主导，其内部结构尚不清楚，可能包含奇点或由其他未知物质组成。'

In [46]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
None: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


"Every cell in our body contains DNA because DNA serves as the fundamental genetic material that carries the instructions necessary for the construction and function of proteins. Proteins are critical components of cell structure and function, and without DNA, cells would lack the ability to produce them, leading to a breakdown in cell activity and organismal life.\n\nDNA replication is essential for cell division, ensuring that each new cell receives a complete set of genetic instructions. This process is semi-conservative, meaning each new DNA molecule consists of one original strand and one newly synthesized strand. This ensures that genetic information is accurately passed on during cell division.\n\nCells maintain their identity through the expression of specific genes, which determines their role in the organism. While all cells share the same DNA, different cells express different genes, leading to the diversity of tissues and organs. DNA's presence in every cell ensures the con